In [6]:
import tensorflow_transform as tft
import tensorflow as tf

In [17]:
def make_tft_serving_input_fn(metadata_dir):
    
    def _input_fn():
    
        # This is what signature data looks like: no feature cross yet
        placeholders = {
            'beta1': tf.placeholder(name='beta1', shape=[None, 1], dtype=tf.float32),
            'beta2': tf.placeholder(name='beta2', shape=[None, 1], dtype=tf.float32),
            'weekday': tf.placeholder(name='weekday', shape=[None, 1], dtype=tf.int64),
            'hour': tf.placeholder(name='hour', shape=[None, 1], dtype=tf.int64)
        }
    
        transform_output = tft.TFTransformOutput(transform_output_dir=metadata_dir)
        features = transform_output.transform_raw_features(placeholders)
            
        return tf.estimator.export.ServingInputReceiver(features, placeholders)

    return _input_fn

In [18]:
sip = make_tft_serving_input_fn('/tmp/tmpyief2v0h/metadata')

In [19]:
receiver = sip()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [20]:
receiver

ServingInputReceiver(features={'weekday': <tf.Tensor 'transform_3/transform/inputs/inputs/weekday_copy:0' shape=(?, 1) dtype=int64>, 'beta2': <tf.Tensor 'transform_3/transform/scale_by_min_max_1/add:0' shape=(?, 1) dtype=float32>, 'hour': <tf.Tensor 'transform_3/transform/inputs/inputs/hour_copy:0' shape=(?, 1) dtype=int64>, 'beta1': <tf.Tensor 'transform_3/transform/scale_by_min_max/add:0' shape=(?, 1) dtype=float32>, 'humidity': <tf.Tensor 'transform_3/transform/inputs/inputs/humidity_copy:0' shape=(?, 1) dtype=float32>}, receiver_tensors={'weekday': <tf.Tensor 'weekday_3:0' shape=(?, 1) dtype=int64>, 'beta2': <tf.Tensor 'beta2_3:0' shape=(?, 1) dtype=float32>, 'hour': <tf.Tensor 'hour_3:0' shape=(?, 1) dtype=int64>, 'beta1': <tf.Tensor 'beta1_3:0' shape=(?, 1) dtype=float32>}, receiver_tensors_alternatives=None)

In [22]:
from training_functions import make_tfr_input_fn
file_pattern = "/tmp/tmpyief2v0h/training.tfr-*"
train_input_fn = make_tfr_input_fn(
    filename_pattern=file_pattern,
    batch_size=1000, 
    options={'num_epochs': None,  # repeat infinitely
             'shuffle_buffer_size': 1000,
             'prefetch_buffer_size': 1000,
             'reader_num_threads': 10,
             'parser_num_threads': 10,
             'sloppy_ordering': True,
             'distribute': False})

In [27]:
input_tensor = train_input_fn()

In [41]:
input_tensor

({'beta1': <tf.Tensor 'IteratorGetNext_1:0' shape=(1000, 1) dtype=float32>,
  'beta2': <tf.Tensor 'IteratorGetNext_1:1' shape=(1000, 1) dtype=float32>,
  'hour': <tf.Tensor 'IteratorGetNext_1:2' shape=(1000, 1) dtype=int64>,
  'weekday': <tf.Tensor 'IteratorGetNext_1:3' shape=(1000, 1) dtype=int64>},
 <tf.Tensor 'IteratorGetNext_1:4' shape=(1000, 1) dtype=float32>)

In [37]:
feed_dict = {
    receiver.receiver_tensors['beta1']: [[1.234]],
    receiver.receiver_tensors['beta2']: [[1.234]],
    receiver.receiver_tensors['weekday']: [[5]],
    receiver.receiver_tensors['hour']: [[16]]
}

In [39]:
feed_dict

{<tf.Tensor 'beta1_3:0' shape=(?, 1) dtype=float32>: [[1.234]],
 <tf.Tensor 'beta2_3:0' shape=(?, 1) dtype=float32>: [[1.234]],
 <tf.Tensor 'hour_3:0' shape=(?, 1) dtype=int64>: [[16]],
 <tf.Tensor 'weekday_3:0' shape=(?, 1) dtype=int64>: [[5]]}

In [40]:
with tf.Session() as sess:
    res = sess.run(receiver, feed_dict=feed_dict)

TypeError: Fetch argument None has invalid type <class 'NoneType'>